In [1]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)

from nnsight import LanguageModel
import random
from datasets import load_dataset
import torch as t
import torch.nn as nn
from dictionary_learning import AutoEncoder
from circuit import get_circuit
from circuit_plotting import plot_circuit
from activation_utils import SparseAct
from tqdm import tqdm
import json
import gc

DEBUGGING = False
if DEBUGGING:
    tracer_kwargs = {'validate' : True, 'scan' : True}
else:
    tracer_kwargs = {'validate' : False, 'scan' : False}

DEVICE = 'cuda:0'


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/yiran/miniconda3/envs/circuit_main/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/yiran/miniconda3/envs/circuit_main/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/yiran/miniconda3/envs/circuit_main/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/yiran/miniconda3/envs/circuit_main/lib/python3.10/site-packages/traitlets/config/

In [2]:
model_cache_dir = "/media/data/yiran/hf_cache/hub"
data_cache_dir = "/media/data/yiran/hf_cache/datasets"
model = LanguageModel('EleutherAI/pythia-70m-deduped', cache_dir=model_cache_dir, device_map=DEVICE, dispatch=True)

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


In [3]:
embed = model.gpt_neox.embed_in
attns = [layer.attention for layer in model.gpt_neox.layers]
mlps = [layer.mlp for layer in model.gpt_neox.layers]
resids = [layer for layer in model.gpt_neox.layers]

In [4]:
all_submods = [embed] + [submod for layer_submods in zip(mlps, attns, resids) for submod in layer_submods]

In [5]:
all_submods

[Embedding(50304, 512),
 GPTNeoXMLP(
   (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
   (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
   (act): GELUActivation()
 ),
 GPTNeoXSdpaAttention(
   (rotary_emb): GPTNeoXRotaryEmbedding()
   (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
   (dense): Linear(in_features=512, out_features=512, bias=True)
   (attention_dropout): Dropout(p=0.0, inplace=False)
 ),
 GPTNeoXLayer(
   (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
   (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
   (post_attention_dropout): Dropout(p=0.0, inplace=False)
   (post_mlp_dropout): Dropout(p=0.0, inplace=False)
   (attention): GPTNeoXSdpaAttention(
     (rotary_emb): GPTNeoXRotaryEmbedding()
     (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
     (dense): Linear(in_features=512, out_features=512, bias=True)
     

In [8]:
is_tuple = {}
with model.trace("_"):
    for submodule in all_submods:
        print(type(submodule.output.shape))
        is_tuple[submodule] = type(submodule.output.shape) == tuple

<class 'torch.Size'>
<class 'torch.Size'>
<class 'tuple'>
<class 'tuple'>
<class 'torch.Size'>
<class 'tuple'>
<class 'tuple'>
<class 'torch.Size'>
<class 'tuple'>
<class 'tuple'>
<class 'torch.Size'>
<class 'tuple'>
<class 'tuple'>
<class 'torch.Size'>
<class 'tuple'>
<class 'tuple'>
<class 'torch.Size'>
<class 'tuple'>
<class 'tuple'>
